In [1]:
import pandas as pd
import numpy as np

In [2]:
# Try reading the file with shift_jis encoding
data_shift_jis = pd.read_csv('../data/在庫推移9月.csv', encoding='shift_jis')
data_shift_jis.head()

,Unnamed: 0,計測日時,品番,拠点所番地,在庫数（箱）,入庫数（箱）,出庫数（箱）
0,0,2023/09/04 00:00:00,019120LC030,30113,12.0,0.0,0.0
1,1,2023/09/04 00:00:00,019120LC040,10114,3.0,0.0,0.0
2,2,2023/09/04 00:00:00,019120LC050,20115,4.0,0.0,0.0
3,3,2023/09/04 00:00:00,019120LC060,30116,9.0,0.0,0.0
4,4,2023/09/04 00:00:00,019120LC070,20117,9.0,0.0,0.0


In [3]:
#以下の手順でデータを処理します：
#計測日時を日付として解釈し、それを基にデータを日付毎にグループ化します。
#各グループ内で、品番毎の入庫数、出庫数を合計します。
#各日付の最後のレコードの在庫数を使用して、その日の在庫数を取得します。

# Convert 計測日時 to datetime and extract only the date
data_shift_jis['計測日'] = pd.to_datetime(data_shift_jis['計測日時']).dt.date

# Group by 計測日 and 品番
grouped = data_shift_jis.groupby(['計測日', '品番'])

# Calculate the sum of 入庫数（箱） and 出庫数（箱） for each group
in_out_sum = grouped[['入庫数（箱）', '出庫数（箱）']].sum()

# Get the last record's 在庫数（箱） for each group (as the end-of-day inventory)
end_of_day_inventory = grouped['在庫数（箱）'].last()

# Combine the results
result = pd.concat([in_out_sum, end_of_day_inventory], axis=1)
result = result.reset_index()

# Extract only the day from the '検収日' column
result['計測日'] = pd.to_datetime(result['計測日']).dt.day

result.columns = ['計測日', '品番', '入庫数（箱）', '出庫数（箱）', '在庫数（箱）']
result.head()

,計測日,品番,入庫数（箱）,出庫数（箱）,在庫数（箱）
0,4,019120LC030,5.0,2.0,15.0
1,4,019120LC040,1.0,0.0,4.0
2,4,019120LC050,1.0,0.0,5.0
3,4,019120LC060,2.0,1.0,10.0
4,4,019120LC070,2.0,1.0,10.0


In [4]:
# Randomly select a product code (品番)
random_product_code = result['品番'].sample(1).iloc[0]
# Extract data for the selected product code
selected_product_data = result[result['品番'] == random_product_code]
selected_product_data

,計測日,品番,入庫数（箱）,出庫数（箱）,在庫数（箱）
134,4,82125ECE020,11.0,27.0,1.0
459,5,82125ECE020,52.0,34.0,9.0
784,6,82125ECE020,42.0,18.0,8.0
1109,7,82125ECE020,59.0,33.0,17.0
1434,8,82125ECE020,40.0,28.0,9.0
1757,11,82125ECE020,24.0,22.0,17.0
2078,12,82125ECE020,33.0,49.0,14.0
2399,13,82125ECE020,43.0,46.0,8.0
2720,14,82125ECE020,51.0,46.0,9.0
3041,15,82125ECE020,54.0,37.0,26.0


In [7]:
# Load the new uploaded file with shift_jis encoding
#kanban_data = pd.read_csv('../data/20231011作成_所在管理LT（9月納入）.csv', encoding='shift_jis')
kanban_data = pd.read_csv('../data/20231006_所在管理LT.csv', encoding='shift_jis')
kanban_data.head()

,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,回収日時,発注取消日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時,長期滞留フラグ
0,1Z23J00061273,XCGG850,10429,35482TFA010,ｸﾗﾝﾌﾟｾﾝｻ-,1000,サトープレス工業（株）,本社工場,回収済,2023/9/26,...,2023/10/6 12:18,< NULL >,0.11,3.93,4.09,9.96,< NULL >,14.01,2023/10/6 12:18,< NULL >
1,1Z23J00090101,XCGP260,10429,35482TFA010,ｸﾗﾝﾌﾟｾﾝｻ-,1000,サトープレス工業（株）,本社工場,順立装置内,2023/9/29,...,< NULL >,< NULL >,0.13,2.26,9.03,< NULL >,< NULL >,< NULL >,2023/10/6 12:11,< NULL >
2,1Z23J00091003,XCGR590,10416,9030106A018,ﾘﾝｸﾞO,2000,ＮＯＫ（株）,< NULL >,順立装置内,2023/9/29,...,< NULL >,< NULL >,0.15,2.27,9.03,< NULL >,< NULL >,< NULL >,2023/10/6 12:11,< NULL >
3,1Z23J00090128,XCGN430,10149,9056450A003,ｼﾑ,100,（株）ムロコーポレーション,< NULL >,回収済,2023/9/27,...,2023/10/6 11:33,< NULL >,0.1,0.86,1.14,8.92,< NULL >,9.22,2023/10/6 11:33,< NULL >
4,1Z23J00018997,XCFD220,10282,9056457A134,ｼﾑ,100,（株）ムロコーポレーション,< NULL >,回収済,2023/9/8,...,2023/10/6 11:33,< NULL >,0.11,< NULL >,1.01,26.98,< NULL >,28.23,2023/10/6 11:33,< NULL >


In [8]:
#手順を以下のように進めます：
#"検収日時（HH:MM）" から日付情報を抽出します。
#品番と検収日でグループ化し、各グループの行数（検収数）をカウントします。

# Convert 検収日時（HH:MM） to datetime and extract only the date
kanban_data['検収日'] = pd.to_datetime(kanban_data['検収日時'], errors='coerce').dt.date

# Group by 検収日 and 品番, then count the number of rows for each group
kanban_grouped = kanban_data.groupby(['検収日', '品番']).size().reset_index(name='検収数')

# Extract only the day from the '検収日' column
kanban_grouped['検収日'] = pd.to_datetime(kanban_grouped['検収日']).dt.day

kanban_grouped.head()

,検収日,品番,検収数
0,1,019128GA010,19
1,1,01912ECB010,6
2,1,01912ECB040,18
3,1,1040043104R,1
4,1,1040052001Z,3


In [9]:
# Extract data for the selected product code
selected_kanban_product_data = kanban_grouped[kanban_grouped['品番'] == random_product_code]
selected_kanban_product_data

,検収日,品番,検収数


In [10]:
# Load the uploaded file with shift_jis encoding
arrangement_data = pd.read_csv('../data/202309_手配数_1Y_12次.csv', encoding='shift_jis')
arrangement_data.head()

,品番,加工図符号,設変符号,品名,ステータス,発注区分,整備室,整備室名,手配区分,工程内外製,...,21(木).1,22(金).1,23(土).1,24(日).1,25(月).1,26(火).1,27(水).1,28(木).1,29(金).1,30(土).1
0,35300-ECB010,NaN,NaN,"PUMP ASSY, OIL W/MOTOR",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,877,815,0,0,877,937,878,938,822,0
1,35580-6GA020,NaN,A,"ACTUATOR ASSY, SHIFT CONTROL",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,0,0,0,0,0,0,0,0,0,0
2,35580-ECB011,NaN,B,"ACTUATOR ASSY, SHIFT CONTROL",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,378,360,0,0,384,420,387,421,354,0
3,1040 052 001Z,NaN,D,"PLUG, W/HEAD STRAIGHT SCREW",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,499,455,0,0,493,517,491,517,468,0
4,1040 183 011P,NaN,C,"BOLT, FLANGE",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,"1,134","1,080",0,0,"1,152","1,260","1,161","1,263","1,062",0


In [133]:
# Correctly format the date columns
date_columns_formatted = [f"{i}({day})" for i, day in zip(range(1, 31), "金土日月火水木" * 5)]
selected_columns_formatted = ['品番'] + date_columns_formatted
subset_data_formatted = arrangement_data[selected_columns_formatted]

# Melt the dataframe to long format
melted_data_formatted = pd.melt(subset_data_formatted, id_vars=['品番'], value_vars=date_columns_formatted, var_name='日付', value_name='日量数')

# Extract only the numeric part for '日付' column
melted_data_formatted['日付'] = melted_data_formatted['日付'].str.extract('(\d+)').astype(int)

# Extract the specified columns
additional_columns = ['当月必要数','当月稼働日数','仕入先名/工場名','箱種類', '収容数', '基準在庫日数', '基準在庫枚数', 'サイクル間隔', 'サイクル回数', 'サイクル情報']
additional_data = arrangement_data[['品番'] + additional_columns]

# Merge the additional data with the melted_data_formatted dataframe on '品番'
merged_data = pd.merge(melted_data_formatted, additional_data, on='品番', how='left')

# Rename the specified columns
merged_data.rename(columns={
    'サイクル間隔': 'A',
    'サイクル回数': 'B',
    'サイクル情報': 'C'
}, inplace=True)

# Convert column to numeric
merged_data['収容数'] = pd.to_numeric(merged_data['収容数'].str.replace(",", ""), errors='coerce')
merged_data['日量数'] = pd.to_numeric(merged_data['日量数'].str.replace(",", ""), errors='coerce')
merged_data['当月必要数'] = merged_data['当月必要数'].str.replace(",", "")
merged_data['当月必要数'] = pd.to_numeric(merged_data['当月必要数'], errors='coerce')
merged_data['当月稼働日数'] = pd.to_numeric(merged_data['当月稼働日数'], errors='coerce')

# Divide the '日量数' column by '収容数' and create a new column '日量数（箱）'
merged_data['日量数（箱）'] = merged_data['日量数'] / merged_data['収容数']

# Replace non-finite values with 0
merged_data['日量数（箱）'] = merged_data['日量数（箱）'].fillna(0)
# Round the '日量数（箱）' column and convert to integer
merged_data['日量数（箱）_切り上げ'] = np.ceil(merged_data['日量数（箱）']).astype(int)

merged_data['平均日量数（箱）'] = (merged_data['当月必要数'] /merged_data['当月稼働日数'])/merged_data['収容数']
# Replace non-finite values with 0
merged_data['平均日量数（箱）'] = merged_data['平均日量数（箱）'].fillna(0)

# Remove "-" and " " from the '品番' column in merged_data
merged_data['品番'] = merged_data['品番'].str.replace("-", "").str.replace(" ", "")

merged_data.head(100)

,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,基準在庫枚数,A,B,C,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）
0,35300ECB010,1,880.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12,0.91,3,1,4,8.04,73.333333,74,74.658730
1,355806GA020,1,0.0,0,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6,0.70,3,1,4,8.04,0.000000,0,0.000000
2,35580ECB011,1,393.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6,0.90,3,1,4,8.04,65.500000,66,65.357143
3,1040052001Z,1,487.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200,0.53,3,1,6,5.76,2.435000,3,2.518571
4,1040183011P,1,1179.0,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.54,1,1,6,5.76,2.358000,3,2.352857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,35145ECB010,1,393.0,8236,21.0,三共鋼業（株）,AW-131 ｾﾝﾖｳ,100,0.62,3,1,1,1.74,3.930000,4,3.921905
96,35145ECE010,1,487.0,10578,21.0,三共鋼業（株）,AW-131 ｾﾝﾖｳ,100,0.65,1,1,1,1.74,4.870000,5,5.037143
97,35195ECE010,1,974.0,21156,21.0,三共鋼業（株）,TP-131 ﾊﾝﾖｳ,400,0.62,3,1,1,1.74,2.435000,3,2.518571
98,35195TFG010,1,974.0,21156,21.0,三共鋼業（株）,TP-131 ﾊﾝﾖｳ,400,0.62,1,1,1,1.74,2.435000,3,2.518571


In [134]:
# Rename '検収日' column to '日付' in selected_kanban_product_data
kanban_grouped = kanban_grouped.rename(columns={'検収日': '日付'})

# Trim whitespace from the '品番' column in both dataframes
kanban_grouped['品番'] = kanban_grouped['品番'].str.strip()

# Merge merged_data and selected_kanban_product_data based on the specified conditions (品番 and 日付)
final_merged_data = pd.merge(merged_data, kanban_grouped, 
                                on=['品番', '日付'], how='inner')

final_merged_data.head()

,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,基準在庫枚数,A,B,C,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）,検収数
0,35300ECB010,1,880.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12,0.91,3,1,4,8.04,73.333333,74,74.658730,55
1,35580ECB011,1,393.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6,0.90,3,1,4,8.04,65.500000,66,65.357143,49
2,1040052001Z,1,487.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200,0.53,3,1,6,5.76,2.435000,3,2.518571,3
3,1040183011P,1,1179.0,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.54,1,1,6,5.76,2.358000,3,2.352857,2
4,3040052001B,1,4007.0,85834,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.52,3,1,6,5.76,8.014000,9,8.174667,6


In [135]:
# Rename '検収日' column to '日付' in selected_kanban_product_data
result = result.rename(columns={'計測日': '日付'})

# Trim whitespace from the '品番' column in both dataframes
result['品番'] = result['品番'].str.strip()

# Merge merged_data and selected_kanban_product_data based on the specified conditions (品番 and 日付)
final_merged_data = pd.merge(final_merged_data, result, 
                                on=['品番', '日付'], how='inner')
final_merged_data.head()

,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,基準在庫枚数,A,B,C,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）,検収数,入庫数（箱）,出庫数（箱）,在庫数（箱）
0,35300ECB010,4,946.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12,0.91,3,1,4,8.04,78.833333,79,74.658730,50,32.0,41.0,127.0
1,35580ECB011,4,411.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6,0.90,3,1,4,8.04,68.500000,69,65.357143,53,27.0,39.0,84.0
2,1040052001Z,4,535.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200,0.53,3,1,6,5.76,2.675000,3,2.518571,1,2.0,1.0,4.0
3,1040183011P,4,1233.0,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.54,1,1,6,5.76,2.466000,3,2.352857,1,1.0,0.0,1.0
4,3040052001B,4,4319.0,85834,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.52,3,1,6,5.76,8.638000,9,8.174667,7,7.0,5.0,5.0


In [136]:
with open('..//data//順立装置の在庫の揺らぎ.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    final_merged_data.to_csv(f)

In [137]:
# Try loading the uploaded file again using the Shift_JIS encoding
df1_new_attempt = pd.read_csv("../data/順立装置の在庫の揺らぎ.csv", encoding='shift_jis')
# Try loading the new uploaded file using the Shift_JIS encoding
df2_new_attempt = pd.read_csv("../data/不等ピッチ情報.csv", encoding='shift_jis')

In [138]:
# Merge df1_new_attempt with df2_new_attempt on the supplier name to add '不等ピッチ係数日' information without dropping columns
df1_updated = pd.merge(df1_new_attempt, df2_new_attempt[['仕入先名', '不等ピッチ係数日']], 
                       left_on='仕入先名/工場名', 
                       right_on='仕入先名', 
                       how='left')

# If '仕入先名' column exists in the merged dataframe, we drop it
if '仕入先名' in df1_updated.columns:
    df1_updated.drop(columns='仕入先名', inplace=True)

df1_updated.head()

,Unnamed: 0,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,...,B,C,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）,検収数,入庫数（箱）,出庫数（箱）,在庫数（箱）,不等ピッチ係数日
0,0,35300ECB010,4,946.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12,0.91,...,4,8.04,78.833333,79,74.658730,50,32.0,41.0,127.0,0.14
1,1,35580ECB011,4,411.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6,0.90,...,4,8.04,68.500000,69,65.357143,53,27.0,39.0,84.0,0.14
2,2,1040052001Z,4,535.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200,0.53,...,6,5.76,2.675000,3,2.518571,1,2.0,1.0,4.0,0.08
3,3,1040183011P,4,1233.0,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.54,...,6,5.76,2.466000,3,2.352857,1,1.0,0.0,1.0,0.08
4,4,3040052001B,4,4319.0,85834,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.52,...,6,5.76,8.638000,9,8.174667,7,7.0,5.0,5.0,0.08


In [139]:
df1_updated['設計値MIN'] = 0.1 * (df1_updated['平均日量数（箱）'] * df1_updated['A'] * (1 +  df1_updated['C'])/ df1_updated['B'])
df1_updated['設計値MAX'] = df1_updated['設計値MIN'] + df1_updated['平均日量数（箱）']/df1_updated['B'] + df1_updated['平均日量数（箱）']*df1_updated['不等ピッチ係数日']
df1_updated.head()

,Unnamed: 0,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,...,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）,検収数,入庫数（箱）,出庫数（箱）,在庫数（箱）,不等ピッチ係数日,設計値MIN,設計値MAX
0,0,35300ECB010,4,946.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12,0.91,...,78.833333,79,74.658730,50,32.0,41.0,127.0,0.14,16.872873,45.989778
1,1,35580ECB011,4,411.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6,0.90,...,68.500000,69,65.357143,53,27.0,39.0,84.0,0.14,14.770714,40.260000
2,2,1040052001Z,4,535.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200,0.53,...,2.675000,3,2.518571,1,2.0,1.0,4.0,0.08,0.283759,0.905007
3,3,1040183011P,4,1233.0,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.54,...,2.466000,3,2.352857,1,1.0,0.0,1.0,0.08,0.265089,0.845460
4,4,3040052001B,4,4319.0,85834,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,0.52,...,8.638000,9,8.174667,7,7.0,5.0,5.0,0.08,0.921012,2.937430


In [140]:
with open('..//data//順立装置の在庫の揺らぎ.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    df1_updated.to_csv(f)

In [141]:
# Shift_JISでファイルを読み込む
data_shiftjis = pd.read_csv('../data/順立装置の在庫の揺らぎ.csv', encoding='shift_jis')

# 最初の5行を表示して内容を確認
data_shiftjis.head()

,Unnamed: 0,Unnamed: 0.1,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,...,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）,検収数,入庫数（箱）,出庫数（箱）,在庫数（箱）,不等ピッチ係数日,設計値MIN,設計値MAX
0,0,0,35300ECB010,4,946.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12,...,78.833333,79,74.658730,50,32.0,41.0,127.0,0.14,16.872873,45.989778
1,1,1,35580ECB011,4,411.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6,...,68.500000,69,65.357143,53,27.0,39.0,84.0,0.14,14.770714,40.260000
2,2,2,1040052001Z,4,535.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200,...,2.675000,3,2.518571,1,2.0,1.0,4.0,0.08,0.283759,0.905007
3,3,3,1040183011P,4,1233.0,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,...,2.466000,3,2.352857,1,1.0,0.0,1.0,0.08,0.265089,0.845460
4,4,4,3040052001B,4,4319.0,85834,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500,...,8.638000,9,8.174667,7,7.0,5.0,5.0,0.08,0.921012,2.937430


In [142]:
# 条件に基づいて新しい列「入庫数（箱）＞検収数」を作成
data_shiftjis['入庫数（箱）＞検収数'] = (data_shiftjis['入庫数（箱）'] > 1.5 * data_shiftjis['検収数']).astype(int)
data_shiftjis['入庫数（箱）＞日量数（箱）'] = (data_shiftjis['入庫数（箱）'] > 1.5 * data_shiftjis['日量数（箱）']).astype(int)
data_shiftjis['入庫数（箱）＜検収数'] = (data_shiftjis['入庫数（箱）']* 1.5 < data_shiftjis['検収数']).astype(int)
data_shiftjis['入庫数（箱）＜日量数（箱）'] = (data_shiftjis['入庫数（箱）'] * 1.5 < data_shiftjis['日量数（箱）']).astype(int)

# 「入庫数（箱）＞検収数」が1の行の数をカウント
count_condition_met = data_shiftjis[data_shiftjis['入庫数（箱）＞検収数'] == 1].shape[0]
count_condition_met2 = data_shiftjis[data_shiftjis['入庫数（箱）＞日量数（箱）'] == 1].shape[0]
count_condition_met3 = data_shiftjis[data_shiftjis['入庫数（箱）＜検収数'] == 1].shape[0]
count_condition_met4 = data_shiftjis[data_shiftjis['入庫数（箱）＜日量数（箱）'] == 1].shape[0]
print(count_condition_met,count_condition_met2,count_condition_met3,count_condition_met4)


229 422 519 612


In [143]:
# 更新されたデータフレームの最初の5行を表示
#data_shiftjis.head()

In [151]:
with open('..//data//順立装置の在庫の揺らぎ（フラグ付き）.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    data_shiftjis.to_csv(f)

In [152]:
import pandas as pd

# 新しいファイルをShift_JISで読み込む
new_data = pd.read_csv('../data/順立装置の在庫の揺らぎ（フラグ付き）.csv', encoding='shift_jis')

X_B = 4
smin = 10
smax = 50

# B=1で収容数が10以下のデータをフィルタリングして、その中で「入庫数（箱）＞検収数」が1のデータの割合を計算
#filtered_data_count = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10)])
#condition_met_count = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10) & (new_data['入庫数（箱）＞検収数'] == 1)])
#condition_met_count2 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10) & (new_data['入庫数（箱）＞日量数（箱）'] == 1)])
#condition_met_count3 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10) & (new_data['入庫数（箱）＜検収数'] == 1)])
#condition_met_count4 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10) & (new_data['入庫数（箱）＜日量数（箱）'] == 1)])

filtered_data_count = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax) ])
condition_met_count = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax) & (new_data['入庫数（箱）＞検収数'] == 1)])
condition_met_count2 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax) & (new_data['入庫数（箱）＞日量数（箱）'] == 1)])
condition_met_count3 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax) & (new_data['入庫数（箱）＜検収数'] == 1)])
condition_met_count4 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax) & (new_data['入庫数（箱）＜日量数（箱）'] == 1)])


proportion_filtered_new = condition_met_count / filtered_data_count if filtered_data_count != 0 else 0
proportion_filtered_new2 = condition_met_count2 / filtered_data_count if filtered_data_count != 0 else 0
proportion_filtered_new3 = condition_met_count3 / filtered_data_count if filtered_data_count != 0 else 0
proportion_filtered_new4 = condition_met_count4 / filtered_data_count if filtered_data_count != 0 else 0
print(proportion_filtered_new,proportion_filtered_new2,proportion_filtered_new3,proportion_filtered_new4)

0.0 0.016666666666666666 0.21666666666666667 0.31666666666666665


In [168]:
import pandas as pd

# 新しいファイルをShift_JISで読み込む
new_data = pd.read_csv('../data/順立装置の在庫の揺らぎ（フラグ付き）.csv', encoding='shift_jis')

X_B = 8
smin = 400
smax = 40000

# B=1で収容数が10以下のデータをフィルタリングして、その中で「入庫数（箱）＞検収数」が1のデータの割合を計算
#filtered_data_count = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10)])
#condition_met_count = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10) & (new_data['入庫数（箱）＞検収数'] == 1)])
#condition_met_count2 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10) & (new_data['入庫数（箱）＞日量数（箱）'] == 1)])
#condition_met_count3 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10) & (new_data['入庫数（箱）＜検収数'] == 1)])
#condition_met_count4 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] <= 10) & (new_data['入庫数（箱）＜日量数（箱）'] == 1)])

filtered_data_count = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax)])
condition_met_count = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax)  & (new_data['在庫数（箱）'] > new_data['設計値MAX']) & (new_data['入庫数（箱）＞検収数'] == 1)])
condition_met_count2 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax) & (new_data['在庫数（箱）'] > new_data['設計値MAX']) & (new_data['入庫数（箱）＞日量数（箱）'] == 1)])
condition_met_count3 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax) & (new_data['在庫数（箱）'] < new_data['設計値MIN']) & (new_data['入庫数（箱）＜検収数'] == 1)])
condition_met_count4 = len(new_data[(new_data['B'] == X_B) & (new_data['収容数'] >= smin) & (new_data['収容数'] < smax) & (new_data['在庫数（箱）'] < new_data['設計値MIN']) & (new_data['入庫数（箱）＜日量数（箱）'] == 1)])

proportion_filtered_new = condition_met_count / filtered_data_count if filtered_data_count != 0 else 0
proportion_filtered_new2 = condition_met_count2 / filtered_data_count if filtered_data_count != 0 else 0
proportion_filtered_new3 = condition_met_count3 / filtered_data_count if filtered_data_count != 0 else 0
proportion_filtered_new4 = condition_met_count4 / filtered_data_count if filtered_data_count != 0 else 0
print(proportion_filtered_new,proportion_filtered_new2,proportion_filtered_new3,proportion_filtered_new4)

0.0 0.0 0.0 0.0
